In [5]:
import pandas as pd 
import requests
import json


In [1]:
def getValues(tagList, start ,end):
        url = "https://data.exactspace.co/kairosapi/api/v1/datapoints/query"
        d = {
            "metrics": [
                {
                    "tags": {},
                    "name": ""
                }
            ],
            "plugins": [],
            "cache_time": 0,
            "cache_time": 0,
            "start_absolute": start,
            "end_absolute": end
        }
        finalDF = pd.DataFrame()
        for tag in tagList:
            d['metrics'][0]['name'] = tag
            res = requests.post(url=url, json=d)
            values = json.loads(res.content)
            df = pd.DataFrame(values["queries"][0]["results"][0]['values'], columns=['time', values["queries"][0]["results"][0]['name']])
            finalDF = pd.concat([finalDF, df], axis=1)

        finalDF = finalDF.loc[:, ~finalDF.columns.duplicated()]
        finalDF.dropna(subset=['time'], inplace=True)
        # finalDF['time'] = pd.to_datetime(finalDF['time'], unit='ms').dt.strftime('%d-%m-%y %H:%M:%S')
        return finalDF

In [8]:
def mld_2_anode_status(start, end):
    tags= [
            'GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric',
            'GAP_GAP03.PLC03.SCHENCK2_FEED_RATE',
            'GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'
        ]
    
    temp_df = pd.DataFrame(columns=['time'])
    temp_df.loc[0, 'time'] = start  # Insert 42 into 'Good' column at index 0
    temp_df.loc[1, 'time'] = end
    temp_df['time'] += (5 * 60 + 30) * 60 * 1000
    temp_df['date'] = pd.to_datetime(temp_df['time'], unit='ms')

    mld_2_df=getValues(tags, start, end)
    
    mld_2_df = mld_2_df.dropna()
    mld_2_df = mld_2_df[(mld_2_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] >= 4000) & (mld_2_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] < 6700) 
        & (mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] >= 1.56) & (mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] <= 1.69)]
    
    mld_2_df = mld_2_df[mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'] % 1 == 0]
    mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'] = mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'].astype(int)

    # Find consecutive duplicate values in the column and mark them for removal
    mld_2_df['to_remove'] = mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'] == mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'].shift(1)

    mld_2_df = mld_2_df[mld_2_df['to_remove'] != True]
    
    mld_2_df.reset_index(drop=True, inplace=True)
    # print(mld_2_df)
    # Assuming df is your DataFrame
    # Convert the 'time' column to datetime format
    mld_2_df['time'] += (5 * 60 + 30) * 60 * 1000
    mld_2_df['date'] = pd.to_datetime(mld_2_df['time'], unit='ms')

    # Define conditions for categorizing 'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'
    condition_better = mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] >= 1.66
    condition_good = (mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] >= 1.65) & (mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] < 1.66)
    condition_bad = mld_2_df['GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric'] < 1.65

    # Create a new column 'category' based on conditions
    mld_2_df['category'] = 'Undefined'
    mld_2_df.loc[condition_better, 'category'] = 'Better'
    mld_2_df.loc[condition_good, 'category'] = 'Good'
    mld_2_df.loc[condition_bad, 'category'] = 'Bad'

    # Create an empty DataFrame with columns
    hourly_counts = pd.DataFrame(columns=['Bad', 'Better', 'Good', 'time'])

    # Group by hour and category, then count occurrences
    hourly_counts = mld_2_df.groupby([mld_2_df['date'].dt.floor('H'), 'category']).size().unstack(fill_value=0)

    complete_hourly_index = pd.date_range(temp_df['date'].min(), temp_df['date'].max(), freq='H')
    reference_df = pd.DataFrame(complete_hourly_index, columns=['date'])

    # Add columns for 'Good', 'Better', and 'Bad' with zeros
    reference_df['Good'] = 0
    reference_df['Better'] = 0
    reference_df['Bad'] = 0

    missing_categories = set(['Bad', 'Better', 'Good']) - set(hourly_counts.columns)
    for category in missing_categories:
            hourly_counts[category] = 0

    hourly_counts = pd.merge(reference_df, hourly_counts, how='left', on=['date'])
    hourly_counts = hourly_counts[['date', 'Good_y', 'Better_y', 'Bad_y']]
    hourly_counts = hourly_counts.rename(columns={'Good_y': 'Good', 'Better_y': 'Better', 'Bad_y':'Bad'})

    hourly_counts = hourly_counts.fillna(0)
    return hourly_counts

In [3]:
start = 1701369000000
end = 1703961000000

In [11]:
hourly_counts = mld_2_anode_status(start, end)

In [18]:
hourly_counts['Bad'].sum()

1015.0

In [20]:
def mld_1_anode_status(start, end):
        tags= [
                'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric',
                'GAP_GAP03.PLC03.SCHENCK2_FEED_RATE',
                'GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'
            ]

        temp_df = pd.DataFrame(columns=['time'])
        temp_df.loc[0, 'time'] = start  # Insert 42 into 'Good' column at index 0
        temp_df.loc[1, 'time'] = end
        temp_df['time'] += (5 * 60 + 30) * 60 * 1000
        temp_df['date'] = pd.to_datetime(temp_df['time'], unit='ms')

        mld_1_df=getValues(tags, start, end)

        mld_1_df = mld_1_df.dropna()
        mld_1_df = mld_1_df[(mld_1_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] >= 4000) & (mld_1_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] < 6700) 
            & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] >= 1.56) & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] <= 1.69)]

        mld_1_df = mld_1_df[mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] % 1 == 0]
        mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].astype(int)

        # Find consecutive duplicate values in the column and mark them for removal
        mld_1_df['to_remove'] = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] == mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].shift(1)

        mld_1_df = mld_1_df[mld_1_df['to_remove'] != True]

        mld_1_df.reset_index(drop=True, inplace=True)


        mld_1_df['time'] += (5 * 60 + 30) * 60 * 1000
        mld_1_df['date'] = pd.to_datetime(mld_1_df['time'], unit='ms')

        # Define conditions for categorizing 'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'
        condition_better = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] >= 1.66
        condition_good = (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] >= 1.65) & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] < 1.66)
        condition_bad = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] < 1.65

        # Create a new column 'category' based on conditions
        mld_1_df['category'] = 'Undefined'
        mld_1_df.loc[condition_better, 'category'] = 'Better'
        mld_1_df.loc[condition_good, 'category'] = 'Good'
        mld_1_df.loc[condition_bad, 'category'] = 'Bad'

        # Create an empty DataFrame with columns
        hourly_counts = pd.DataFrame(columns=['Bad', 'Better', 'Good', 'time'])

        # Group by hour and category, then count occurrences
        hourly_counts = mld_1_df.groupby([mld_1_df['date'].dt.floor('H'), 'category']).size().unstack(fill_value=0)

        complete_hourly_index = pd.date_range(temp_df['date'].min(), temp_df['date'].max(), freq='H')
        reference_df = pd.DataFrame(complete_hourly_index, columns=['date'])

        # Add columns for 'Good', 'Better', and 'Bad' with zeros
        reference_df['Good'] = 0
        reference_df['Better'] = 0
        reference_df['Bad'] = 0

        missing_categories = set(['Bad', 'Better', 'Good']) - set(hourly_counts.columns)
        for category in missing_categories:
                hourly_counts[category] = 0

        hourly_counts = pd.merge(reference_df, hourly_counts, how='left', on=['date'])
        hourly_counts = hourly_counts[['date', 'Good_y', 'Better_y', 'Bad_y']]
        hourly_counts = hourly_counts.rename(columns={'Good_y': 'Good', 'Better_y': 'Better', 'Bad_y':'Bad'})

        hourly_counts = hourly_counts.fillna(0)
        return hourly_counts

In [21]:
hourly_counts = mld_1_anode_status(start, end)

In [22]:
hourly_counts.head()

,date,Good,Better,Bad
0,2023-12-01 00:00:00,13.0,4.0,0.0
1,2023-12-01 01:00:00,8.0,7.0,0.0
2,2023-12-01 02:00:00,13.0,4.0,0.0
3,2023-12-01 03:00:00,11.0,5.0,0.0
4,2023-12-01 04:00:00,8.0,4.0,5.0


In [23]:
hourly_counts.tail()

,date,Good,Better,Bad
716,2023-12-30 20:00:00,0.0,0.0,0.0
717,2023-12-30 21:00:00,0.0,0.0,0.0
718,2023-12-30 22:00:00,0.0,0.0,0.0
719,2023-12-30 23:00:00,0.0,0.0,0.0
720,2023-12-31 00:00:00,0.0,0.0,0.0


In [26]:
hourly_counts['Bad'].sum()

1184.0

In [27]:
def kpis_dashboard(start, end):
        tags= [
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric',
        'GAP_GAP03.PLC03.SCHENCK2_FEED_RATE',
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Number',
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Weight',
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Height',
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Dry_Density'
        ]

        temp_df = pd.DataFrame(columns=['time'])
        temp_df.loc[0, 'time'] = start  # Insert 42 into 'Good' column at index 0
        temp_df.loc[1, 'time'] = end
        temp_df['time'] += (5 * 60 + 30) * 60 * 1000
        temp_df['date'] = pd.to_datetime(temp_df['time'], unit='ms')

        mld_1_df=getValues(tags, start, end)

        mld_1_df = mld_1_df.dropna()
        mld_1_df = mld_1_df[(mld_1_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] >= 4000) & (mld_1_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] < 6700) 
        & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] >= 1.56) & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] <= 1.69)]

        mld_1_df = mld_1_df[mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] % 1 == 0]
        mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].astype(int)

        # Find consecutive duplicate values in the column and mark them for removal
        mld_1_df['to_remove'] = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] == mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].shift(1)

        mld_1_df = mld_1_df[mld_1_df['to_remove'] != True]

        mld_1_df.reset_index(drop=True, inplace=True)


        mld_1_df['time'] += (5 * 60 + 30) * 60 * 1000
        mld_1_df['date'] = pd.to_datetime(mld_1_df['time'], unit='ms')
        mld_1_df.set_index('date', inplace=True)


        columns_of_interest = [
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Weight',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Dry_Density',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Height'
        ]

        # Calculate hourly mean for multiple columns
        hourly_mean = mld_1_df[columns_of_interest].resample('H').mean()
        hourly_std = mld_1_df[columns_of_interest].resample('H').std()
        hourly_std = hourly_std.round(3)
        hourly_mean = hourly_mean.round(3)

        hourly_std.reset_index(inplace=True)
        hourly_mean.reset_index(inplace=True)


        complete_hourly_index = pd.date_range(temp_df['date'].min(), temp_df['date'].max(), freq='H')
        reference_df = pd.DataFrame(complete_hourly_index, columns=['date'])

        hourly_mean = pd.merge(reference_df, hourly_mean, how='left', on=['date'])
        hourly_std = pd.merge(reference_df, hourly_std, how='left', on=['date'])
        global test_df
        test_df = hourly_std.copy()

        missing_categories1 = set(['GAP_GAP04.PLC04.MLD1_DATA_Anode_Weight',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Dry_Density',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Height']) - set(hourly_mean.columns)

        missing_categories2 = set(['GAP_GAP04.PLC04.MLD1_DATA_Anode_Weight',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Dry_Density',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric',
            'GAP_GAP04.PLC04.MLD1_DATA_Anode_Height']) - set(hourly_std.columns)

        for category in missing_categories1:
                hourly_mean[category] = 0

        for category in missing_categories2:
                hourly_std[category] = 0

        hourly_mean = hourly_mean.fillna(0)
        hourly_std = hourly_std.fillna(0)

        hourly_mean.set_index('date', inplace=True)
        hourly_std.set_index('date', inplace=True)

        # Convert the index to epoch format
        hourly_mean["time"] = hourly_mean.index.astype(int) // 10**6  
        hourly_mean["time"] -= (5 * 60 + 30) * 60 * 1000

        hourly_std["time"] = hourly_std.index.astype(int) // 10**6  
        hourly_std["time"] -= (5 * 60 + 30) * 60 * 1000

        return hourly_mean, hourly_std

In [28]:
hourly_means, hourly_std = kpis_dashboard(start, end)

c:\Users\vikra\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:87: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
c:\Users\vikra\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:90: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.


In [45]:
hourly_std[hourly_std['GAP_GAP04.PLC04.MLD1_DATA_Anode_Weight'] > 0].mean()

GAP_GAP04.PLC04.MLD1_DATA_Anode_Weight         9.619089e+00
GAP_GAP04.PLC04.MLD1_DATA_Anode_Dry_Density    6.083784e-03
GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric      5.670270e-03
GAP_GAP04.PLC04.MLD1_DATA_Anode_Height         4.329105e+00
time                                           1.702569e+12
dtype: float64

In [53]:
def process_parameters(start, end):

        tags= [
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric',
        'GAP_GAP03.PLC03.SCHENCK2_FEED_RATE',
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Number',
        'GAP_GAP01.PLC01.U362_E020_MVF_01_ACTRL_AUTOSPEEDREF',
        'GAP_GAP01.PLC01._362_E020_ZT_01.PV',
        'GAP_GAP03.PLC03.J362_J150_JT_01_PW01_IN',
        'GAP_GAP04.PLC04.EU2_DATA_Cooler_Load',
        'GAP_GAP03.PLC03.J362_J155_JT_01_PW01_IN',
        'GAP_GAP04.PLC04.MLD2_DATA_Paste_Eval_Temp',
        'GAP_GAP04.PLC04.K363_K040A_MVF_01_VTK',
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Vaccum_Pres',
        'GAP_GAP04.PLC04.MLD2_DATA_Anode_Counter_Pres',
        'GAP_GAP03.PLC03.U362_J130_TIT_01_PV',
        'GAP_GAP02.PLC02._362_H120_TIT_01.PV',
        'GAP_GAP03.PLC03._362_J150_WIT_01.PV'
        ]

        temp_df = pd.DataFrame(columns=['time'])
        temp_df.loc[0, 'time'] = start  
        temp_df.loc[1, 'time'] = end
        temp_df['time'] += (5 * 60 + 30) * 60 * 1000
        temp_df['date'] = pd.to_datetime(temp_df['time'], unit='ms')

        mld_1_df=getValues(tags, start, end)

        mld_1_df = mld_1_df.dropna()
        mld_1_df = mld_1_df[(mld_1_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] >= 4000) & (mld_1_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] < 6700) 
        & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] >= 1.56) & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] <= 1.69)]

        mld_1_df = mld_1_df[mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] % 1 == 0]
        mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].astype(int)
        # Find consecutive duplicate values in the column and mark them for removal
        mld_1_df['to_remove'] = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] == mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].shift(1)

        mld_1_df = mld_1_df[mld_1_df['to_remove'] != True]

        mld_1_df.reset_index(drop=True, inplace=True)


        mld_1_df['time'] += (5 * 60 + 30) * 60 * 1000
        mld_1_df['date'] = pd.to_datetime(mld_1_df['time'], unit='ms')
        mld_1_df.set_index('date', inplace=True)


        columns_of_interest = [
        'GAP_GAP01.PLC01.U362_E020_MVF_01_ACTRL_AUTOSPEEDREF',
        'GAP_GAP01.PLC01._362_E020_ZT_01.PV',
        'GAP_GAP03.PLC03.J362_J150_JT_01_PW01_IN',
        'GAP_GAP04.PLC04.EU2_DATA_Cooler_Load',
        'GAP_GAP03.PLC03.J362_J155_JT_01_PW01_IN',
        'GAP_GAP04.PLC04.MLD2_DATA_Paste_Eval_Temp',
        'GAP_GAP04.PLC04.K363_K040A_MVF_01_VTK',
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Vaccum_Pres',
        'GAP_GAP04.PLC04.MLD2_DATA_Anode_Counter_Pres',
        'GAP_GAP03.PLC03.U362_J130_TIT_01_PV',
        'GAP_GAP02.PLC02._362_H120_TIT_01.PV',
        'GAP_GAP03.PLC03._362_J150_WIT_01.PV'
        ]

        # Calculate hourly mean for multiple columns
        hourly_mean = mld_1_df[columns_of_interest].resample('H').mean()

        hourly_mean = hourly_mean.round(3)

        hourly_mean.reset_index(inplace=True)


        complete_hourly_index = pd.date_range(temp_df['date'].min(), temp_df['date'].max(), freq='H')
        reference_df = pd.DataFrame(complete_hourly_index, columns=['date'])

        hourly_mean = pd.merge(reference_df, hourly_mean, how='left', on=['date'])

        missing_categories = set(['GAP_GAP01.PLC01.U362_E020_MVF_01_ACTRL_AUTOSPEEDREF',
        'GAP_GAP01.PLC01._362_E020_ZT_01.PV',
        'GAP_GAP03.PLC03.J362_J150_JT_01_PW01_IN',
        'GAP_GAP04.PLC04.EU2_DATA_Cooler_Load',
        'GAP_GAP03.PLC03.J362_J155_JT_01_PW01_IN',
        'GAP_GAP04.PLC04.MLD2_DATA_Paste_Eval_Temp',
        'GAP_GAP04.PLC04.K363_K040A_MVF_01_VTK',
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Vaccum_Pres',
        'GAP_GAP04.PLC04.MLD2_DATA_Anode_Counter_Pres',
        'GAP_GAP03.PLC03.U362_J130_TIT_01_PV',
        'GAP_GAP02.PLC02._362_H120_TIT_01.PV',
        'GAP_GAP03.PLC03._362_J150_WIT_01.PV']) - set(hourly_mean.columns)

        for category in missing_categories:
                hourly_mean[category] = 0
        hourly_mean = hourly_mean.fillna(0)
        return hourly_mean

In [54]:
hourly_means = process_parameters(start, end)

In [57]:
hourly_means[hourly_means['GAP_GAP01.PLC01.U362_E020_MVF_01_ACTRL_AUTOSPEEDREF'] > 0].mean()

c:\Users\vikra\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """Entry point for launching an IPython kernel.


GAP_GAP01.PLC01.U362_E020_MVF_01_ACTRL_AUTOSPEEDREF      55.181419
GAP_GAP01.PLC01._362_E020_ZT_01.PV                       74.150357
GAP_GAP03.PLC03.J362_J150_JT_01_PW01_IN                  98.060344
GAP_GAP04.PLC04.EU2_DATA_Cooler_Load                      0.000000
GAP_GAP03.PLC03.J362_J155_JT_01_PW01_IN                  96.608094
GAP_GAP04.PLC04.MLD2_DATA_Paste_Eval_Temp               175.549602
GAP_GAP04.PLC04.K363_K040A_MVF_01_VTK                    52.679685
GAP_GAP04.PLC04.MLD1_DATA_Anode_Vaccum_Pres              98.637076
GAP_GAP04.PLC04.MLD2_DATA_Anode_Counter_Pres              1.324250
GAP_GAP03.PLC03.U362_J130_TIT_01_PV                     179.390307
GAP_GAP02.PLC02._362_H120_TIT_01.PV                     233.479034
GAP_GAP03.PLC03._362_J150_WIT_01.PV                    2532.920021
dtype: float64

In [ ]:
gap_anode_rhodax_speed_mean = hourly_mean[['time', 'GAP_GAP01.PLC01.U362_E020_MVF_01_ACTRL_AUTOSPEEDREF']].values.tolist()
gap_anode_Rhodax_gap_mean = hourly_mean[['time', 'GAP_GAP01.PLC01._362_E020_ZT_01.PV']].values.tolist()
gap_anode_Cooler_Load_mean = hourly_mean[['time', 'GAP_GAP04.PLC04.EU2_DATA_Cooler_Load']].values.tolist()
gap_anode_Cooler_Rotor_Power_mean = hourly_mean[['time', 'GAP_GAP03.PLC03.J362_J155_JT_01_PW01_IN']].values.tolist()
gap_anode_Paste_Temp_mean = hourly_mean[['time', 'GAP_GAP04.PLC04.MLD2_DATA_Paste_Eval_Temp']].values.tolist()
gap_anode_Vibration_Time_mean = hourly_mean[['time', 'GAP_GAP04.PLC04.K363_K040A_MVF_01_VTK']].values.tolist()
gap_anode_Counter_Pressure_mean = hourly_mean[['time', 'GAP_GAP04.PLC04.MLD2_DATA_Anode_Counter_Pres']].values.tolist()
gap_anode_vacum_pressure_mean = hourly_mean[['time', 'GAP_GAP04.PLC04.MLD1_DATA_Anode_Vaccum_Pres']].values.tolist()
gap_anode_PH_Screw_Temp_mean = hourly_mean[['time', 'GAP_GAP03.PLC03.U362_J130_TIT_01_PV']].values.tolist()
gap_anode_Pitch_Temp_mean = hourly_mean[['time', 'GAP_GAP02.PLC02._362_H120_TIT_01.PV']].values.tolist()
gap_anode_Mixer_Load_mean = hourly_mean[['time', 'GAP_GAP03.PLC03._362_J150_WIT_01.PV']].values.tolist()
gap_anode_Mixer_Rotor_Power_mean = hourly_mean[['time', 'GAP_GAP03.PLC03.J362_J150_JT_01_PW01_IN']].values.tolist()

In [65]:
def key_parameters(start, end):
        tags= [
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric',
        'GAP_GAP03.PLC03.SCHENCK2_FEED_RATE',
        'GAP_GAP04.PLC04.MLD1_DATA_Anode_Number',
        'GAP_GAP03.PLC03.ACTUAL_FORMULA.KGS',
        'GAP_GAP03.PLC03.ACTUAL_FORMULA.KLP',
        'GAP_GAP04.PLC04.U363_K145_FIT_01_PV',
        'GAP_GAP03.PLC03.ACTUAL_FORMULA.KFR',
        'GAP_GAP03.PLC03.ACTUAL_FORMULA_KBS'
        ]

        temp_df = pd.DataFrame(columns=['time'])
        temp_df.loc[0, 'time'] = start  
        temp_df.loc[1, 'time'] = end
        temp_df['time'] += (5 * 60 + 30) * 60 * 1000
        temp_df['date'] = pd.to_datetime(temp_df['time'], unit='ms')

        mld_1_df=getValues(tags, start, end)

        mld_1_df = mld_1_df.dropna()
        mld_1_df = mld_1_df[(mld_1_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] >= 4000) & (mld_1_df['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE'] < 6700) 
        & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] >= 1.56) & (mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric'] <= 1.69)]

        mld_1_df = mld_1_df[mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] % 1 == 0]
        mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].astype(int)

        # Find consecutive duplicate values in the column and mark them for removal
        mld_1_df['to_remove'] = mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] == mld_1_df['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].shift(1)

        mld_1_df = mld_1_df[mld_1_df['to_remove'] != True]

        mld_1_df.reset_index(drop=True, inplace=True)


        mld_1_df['time'] += (5 * 60 + 30) * 60 * 1000
        mld_1_df['date'] = pd.to_datetime(mld_1_df['time'], unit='ms')
        mld_1_df.set_index('date', inplace=True)


        columns_of_interest = [
        'GAP_GAP03.PLC03.ACTUAL_FORMULA.KGS',
        'GAP_GAP03.PLC03.ACTUAL_FORMULA.KLP',
        'GAP_GAP04.PLC04.U363_K145_FIT_01_PV',
        'GAP_GAP03.PLC03.ACTUAL_FORMULA.KFR',
        'GAP_GAP03.PLC03.ACTUAL_FORMULA_KBS'
        ]

        # Calculate hourly mean for multiple columns
        hourly_mean = mld_1_df[columns_of_interest].resample('H').mean()

        hourly_mean = hourly_mean.round(3)

        hourly_mean.reset_index(inplace=True)


        complete_hourly_index = pd.date_range(temp_df['date'].min(), temp_df['date'].max(), freq='H')
        reference_df = pd.DataFrame(complete_hourly_index, columns=['date'])

        hourly_mean = pd.merge(reference_df, hourly_mean, how='left', on=['date'])

        missing_categories = set(['GAP_GAP03.PLC03.ACTUAL_FORMULA.KGS',
        'GAP_GAP03.PLC03.ACTUAL_FORMULA.KLP',
        'GAP_GAP04.PLC04.U363_K145_FIT_01_PV',
        'GAP_GAP03.PLC03.ACTUAL_FORMULA.KFR',
        'GAP_GAP03.PLC03.ACTUAL_FORMULA_KBS']) - set(hourly_mean.columns)

        for category in missing_categories:
                hourly_mean[category] = 0

        hourly_mean = hourly_mean.fillna(0)
        
        return hourly_mean

In [66]:
hourly_means = key_parameters(start, end)

In [67]:
hourly_means.head()

,date,GAP_GAP03.PLC03.ACTUAL_FORMULA.KGS,GAP_GAP03.PLC03.ACTUAL_FORMULA.KLP,GAP_GAP04.PLC04.U363_K145_FIT_01_PV,GAP_GAP03.PLC03.ACTUAL_FORMULA.KFR,GAP_GAP03.PLC03.ACTUAL_FORMULA_KBS
0,2023-12-01 00:00:00,3.0,12.850,3.496,20.0,25.0
1,2023-12-01 01:00:00,3.0,12.829,3.474,20.0,25.0
2,2023-12-01 02:00:00,3.0,12.850,0.173,20.0,25.0
3,2023-12-01 03:00:00,3.0,12.850,0.152,20.0,25.0
4,2023-12-01 04:00:00,3.0,12.850,0.282,20.0,25.0


In [68]:
hourly_means[hourly_means['GAP_GAP03.PLC03.ACTUAL_FORMULA.KGS'] > 0].mean()

c:\Users\vikra\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """Entry point for launching an IPython kernel.


GAP_GAP03.PLC03.ACTUAL_FORMULA.KGS      3.000000
GAP_GAP03.PLC03.ACTUAL_FORMULA.KLP     12.985118
GAP_GAP04.PLC04.U363_K145_FIT_01_PV     0.423235
GAP_GAP03.PLC03.ACTUAL_FORMULA.KFR     20.000000
GAP_GAP03.PLC03.ACTUAL_FORMULA_KBS     24.361810
dtype: float64

In [ ]:
gap_anode_fines_mean = hourly_mean[['time', 'GAP_GAP03.PLC03.ACTUAL_FORMULA.KFR']].values.tolist()
gap_anode_paste_rejection_mean = hourly_mean[['time', 'GAP_GAP04.PLC04.U363_K145_FIT_01_PV']].values.tolist()
gap_anode_pitch_mean = hourly_mean[['time', 'GAP_GAP03.PLC03.ACTUAL_FORMULA.KLP']].values.tolist()
gap_anode_green_scrap_mean = hourly_mean[['time', 'GAP_GAP03.PLC03.ACTUAL_FORMULA.KGS']].values.tolist()
gap_anode_green_butt_mean = hourly_mean[['time', 'GAP_GAP03.PLC03.ACTUAL_FORMULA_KBS']].values.tolist()